# IEOR 4004 Group Project1: 

## Motivation:

- determining the minimum amount of funding (in total) needed to meet their target for each area for childcare facility expansion
- facilities: loc, cap

In [ ]:
import haversine
from haversine import haversine, Unit
def calc_distance(lat1, lon1, lat2, lon2): 
    # Coordinates in decimal degrees
    lat1, lon1 = 40.807384, -73.963036  # Columbia University
    lat2, lon2 = 40.8182, -73.9597  # Columbia Business School (New)
    # Calculate the distance in kilometers
    distance = haversine((lat1, lon1), (lat2, lon2), unit=Unit.MILES)
    print(f"The distance between the two points is {distance} mile")

In [33]:
import pandas as pd


income = pd.read_csv("avg_individual_income.csv")
employment = pd.read_csv("employment_rate.csv")
pop = pd.read_csv("population.csv")
#假设各个年龄段分布均匀
pop['10-12'] = pop['10-14']*3/5
fac = pd.read_csv("child_care_regulated.csv")

zip = income.merge(employment,how="left",left_on="ZIP code", right_on="zipcode")
zip['is_high_demand'] = (zip['employment rate']>=0.6) | ((zip['average income'])<60000)


slots = fac.groupby("zip_code").sum()[['infant_capacity','toddler_capacity','preschool_capacity','school_age_capacity','children_capacity','total_capacity']]


zip = zip.merge(slots,how="left",left_on="ZIP code", right_on="zip_code")
zip = zip.merge(pop[['zipcode','-5','5-9','10-12']],left_on="ZIP code", right_on="zipcode")
zip['is_desert'] = (zip['is_high_demand'] &  (zip['total_capacity'] <= 1/2*(zip['-5']+zip['5-9']+zip['10-12']))) | (zip['is_high_demand'] & (zip['total_capacity'] <= 1/3*(zip['-5']+zip['5-9']+zip['10-12'])))
zip
zip.head(10)


,ZIP code,average income,zipcode_x,employment rate,is_high_demand,infant_capacity,toddler_capacity,preschool_capacity,school_age_capacity,children_capacity,total_capacity,zipcode_y,-5,5-9,10-12,is_desert
0,10001,102878.033603,10001.0,0.595097,False,0.0,0.0,0.0,585.0,24.0,609.0,10001,744,784,565.2,False
1,10002,59604.041165,10002.0,0.520662,True,0.0,0.0,18.0,4508.0,203.0,4729.0,10002,2142,3046,1918.8,False
2,10003,114273.049645,10003.0,0.497244,False,0.0,0.0,0.0,1995.0,0.0,1995.0,10003,1440,1034,571.8,False
3,10004,132004.310345,10004.0,0.506661,False,0.0,0.0,0.0,263.0,0.0,263.0,10004,433,182,96.6,False
4,10005,121437.713311,10005.0,0.665833,True,0.0,0.0,0.0,39.0,0.0,39.0,10005,484,204,137.4,True
5,10006,126377.118644,10006.0,0.631692,True,0.0,0.0,14.0,142.0,0.0,156.0,10006,128,96,45.0,False
6,10007,138853.904282,10007.0,0.528910,False,0.0,0.0,0.0,284.0,0.0,284.0,10007,605,451,174.0,False
7,10009,77133.233533,10009.0,0.514567,False,0.0,0.0,18.0,1660.0,106.0,1784.0,10009,1896,1658,1101.0,False
8,10010,116272.698810,10010.0,0.492749,False,0.0,0.0,0.0,234.0,0.0,234.0,10010,1422,1592,568.8,False
9,10011,120420.792079,10011.0,0.557000,False,0.0,0.0,42.0,1908.0,6.0,1956.0,10011,1209,1200,818.4,False


In [34]:
fac = fac.merge(pop,how="left",left_on="zip_code", right_on="zipcode")
fac


In [31]:
#第二题用的


def cost_func(existing,expands):
    if expands/existing < 0.1:
        cost = 20000 + 200*expands
    elif 0.1 < expands/existing < 0.15:
        cost = 20000 + 200*0.1*existing + (expands-0.1*existing)*400
    elif 0.15 < expands/existing < 0.2:
        cost = 20000 + 200*0.1*existing + 400*0.05*existing +(expands-0.15*existing)*1000

    return cost

print(cost_func(1000,160))
    
    

70000.0
